In [10]:
from collections import Counter
from tqdm import tqdm
import json
import os
from typing import List, Any
from indicnlp.tokenize.indic_tokenize import trivial_tokenize_indic

In [2]:
with open('data/Odia_structured_wordlist.json', 'r', encoding='utf-8') as f:
    di = json.load(f)

In [4]:
di[list(di.keys())[4]]

{'gender': None,
 'pronunciation': 'Haḻka pāibā',
 'raw': ' (ଅମୁକର) ହଳକ ପାଇବା— Haḻka pāibā [synonym(s): হলক পাত্তয়া हलक पाना] ଦେ. ବି— 1। [କୌଣସି କାର୍ଯ୍ୟ କରିବାପାଇଁ] ସାହସ ଉପୁଜିବା। 1. Having the boldness or courage (to undertake a particular work). (ଯଥା—ଏଡ଼େ ଡେଙ୍ଗା ଗଛରେ ଚଢ଼ିବାକୁ ମୋର ହଲକ୍ ପାଉ ନାହିଁ।) ',
 'synonyms': '[synonym(s): হলক পাত্তয়া हलक पाना]',
 'word_details': [{'juktakhyara': 'ଦେ. ବି',
   'meaning': '1। [କୌଣସି କାର୍ଯ୍ୟ କରିବାପାଇଁ] ସାହସ ଉପୁଜିବା। 1. Having the boldness or courage (to undertake a particular work). (ଯଥା—ଏଡ଼େ ଡେଙ୍ଗା ଗଛରେ ଚଢ଼ିବାକୁ ମୋର ହଲକ୍ ପାଉ ନାହିଁ।)',
   'verse': None}]}

In [6]:

filename = os.path.join('data/or')
assert os.path.isfile(filename)  # sanity check

In [7]:
with open(filename, 'r', encoding='utf-8') as f:
    lines = [s.strip() for s in tqdm(f.readlines(), desc='read lines from file')]

read lines from file: 100%|██████████| 3594672/3594672 [00:02<00:00, 1236541.43it/s]


In [ ]:
def tokenize_text(text: List[str]) -> List[List[str]]:
    """Tokenize text"""
    return [trivial_tokenize_indic(sent) for sent in tqdm(text, desc='tokenize', unit=' sentences')]

In [ ]:
# tokenize
tokens = tokenize_text(lines)